# 02 - Transformação e Curadoria dos Dados (Camada Silver)

## Objetivo da Camada Silver

A camada Silver tem como objetivo realizar a **limpeza, padronização e curadoria** dos dados provenientes da camada Bronze, preparando-os para análises mais confiáveis e para a modelagem analítica na camada Gold (Data Warehouse).

Nesta etapa, são realizadas transformações como:
- Padronização de tipos de dados
- Validação de domínios
- Tratamento de valores nulos
- Organização semântica das variáveis

O resultado desta etapa é um conjunto de dados consistente, estruturado e adequado para análises exploratórias e modelagem dimensional.


In [0]:
# Leitura da camada Bronze
df_bronze = spark.table("maintenance_bronze")

# Visualização inicial
df_bronze.show(5)

# Verificação do schema
df_bronze.printSchema()


+---+----------+----+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+
|UDI|Product_ID|Type|Air_temperature_K|Process_temperature_K|Rotational_speed_rpm|Torque_Nm|Tool_wear_min|Machine_failure|TWF|HDF|PWF|OSF|RNF|
+---+----------+----+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+
|  1|    M14860|   M|            298.1|                308.6|                1551|     42.8|            0|              0|  0|  0|  0|  0|  0|
|  2|    L47181|   L|            298.2|                308.7|                1408|     46.3|            3|              0|  0|  0|  0|  0|  0|
|  3|    L47182|   L|            298.1|                308.5|                1498|     49.4|            5|              0|  0|  0|  0|  0|  0|
|  4|    L47183|   L|            298.2|                308.6|                1433|     39.5|            7|              0|  0|  0|  0|  0|  0|

A leitura da camada Bronze foi realizada com o objetivo de validar a disponibilidade da tabela e a estrutura das colunas, garantindo que os dados necessários para a etapa de tratamento e curadoria estivessem corretamente antes do início das transformações na camada Silver.


In [0]:
from pyspark.sql.functions import col

df_silver = df_bronze.select(
    col("UDI").cast("long").alias("udi"),
    col("Product_ID").cast("string").alias("product_id"),
    col("Type").cast("string").alias("type"),
    col("Air_temperature_K").cast("double").alias("air_temperature_k"),
    col("Process_temperature_K").cast("double").alias("process_temperature_k"),
    col("Rotational_speed_rpm").cast("long").alias("rotational_speed_rpm"),
    col("Torque_Nm").cast("double").alias("torque_nm"),
    col("Tool_wear_min").cast("long").alias("tool_wear_min"),
    col("Machine_failure").cast("long").alias("machine_failure"),
    col("TWF").cast("long").alias("twf"),
    col("HDF").cast("long").alias("hdf"),
    col("PWF").cast("long").alias("pwf"),
    col("OSF").cast("long").alias("osf"),
    col("RNF").cast("long").alias("rnf")
)

df_silver.printSchema()


root
 |-- udi: long (nullable = true)
 |-- product_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- air_temperature_k: double (nullable = true)
 |-- process_temperature_k: double (nullable = true)
 |-- rotational_speed_rpm: long (nullable = true)
 |-- torque_nm: double (nullable = true)
 |-- tool_wear_min: long (nullable = true)
 |-- machine_failure: long (nullable = true)
 |-- twf: long (nullable = true)
 |-- hdf: long (nullable = true)
 |-- pwf: long (nullable = true)
 |-- osf: long (nullable = true)
 |-- rnf: long (nullable = true)



Esta transformação foi realizada com o objetivo de padronizar os dados da camada Bronze, alinhando-os às boas práticas de engenharia de dados adotadas na camada Silver. Foram aplicadas padronizações nos nomes das colunas, utilizando convenção `snake_case`, bem como ajustes explícitos de tipos de dados, garantindo consistência, clareza semântica e facilidade de uso em consultas SQL e análises posteriores.

Embora o Delta Lake seja capaz de armazenar os dados sem essas alterações, a padronização nesta etapa é fundamental para assegurar governança, manutenibilidade e compatibilidade com a modelagem dimensional e com as análises de negócio realizadas nas etapas seguintes do pipeline.

In [0]:
from pyspark.sql.functions import sum, when

df_silver.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df_silver.columns
]).show()


+---+----------+----+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+
|udi|product_id|type|air_temperature_k|process_temperature_k|rotational_speed_rpm|torque_nm|tool_wear_min|machine_failure|twf|hdf|pwf|osf|rnf|
+---+----------+----+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+
|  0|         0|   0|                0|                    0|                   0|        0|            0|              0|  0|  0|  0|  0|  0|
+---+----------+----+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+



Foi realizada a verificação de valores nulos por coluna na camada Silver, com o objetivo de garantir a que nenhum dado esteja faltando antes da modelagem analítica.


In [0]:
# Criando uma view temporária para uso em SQL
df_silver.createOrReplaceTempView("vw_maintenance_silver")


> Para a realização das análises de qualidade utilizando SQL, foi criada uma view temporária a partir do DataFrame da camada Silver, permitindo a integração entre PySpark e SQL no ambiente Databricks.

In [0]:
%sql
SELECT
  MIN(air_temperature_k) AS min_air_temp,
  MAX(air_temperature_k) AS max_air_temp,
  MIN(process_temperature_k) AS min_process_temp,
  MAX(process_temperature_k) AS max_process_temp,
  MIN(rotational_speed_rpm) AS min_rpm,
  MAX(rotational_speed_rpm) AS max_rpm,
  MIN(torque_nm) AS min_torque,
  MAX(torque_nm) AS max_torque,
  MIN(tool_wear_min) AS min_tool_wear,
  MAX(tool_wear_min) AS max_tool_wear
FROM vw_maintenance_silver;


min_air_temp,max_air_temp,min_process_temp,max_process_temp,min_rpm,max_rpm,min_torque,max_torque,min_tool_wear,max_tool_wear
295.3,304.5,305.7,313.8,1168,2886,3.8,76.6,0,253


## Análise da Qualidade dos Dados

A análise da qualidade dos dados indicou que o conjunto apresenta alta consistência, com ausência ou quantidade insignificante de valores nulos nas variáveis analisadas.

Os domínios observados para variáveis contínuas como temperatura, torque, velocidade de rotação e desgaste da ferramenta encontram-se dentro de faixas esperadas para operações industriais, não sendo identificados valores extremos que comprometam a análise.

Dessa forma, os dados da camada Silver foram considerados aptos para utilização na etapa de modelagem dimensional e análises de negócio.


In [0]:
# Escrita da camada Silver como tabela Delta gerenciada
df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("maintenance_silver")


Os dados tratados da camada Silver foram persistidos como tabela Delta gerenciada, garantindo reexecução do pipeline e disponibilidade para análises e modelagem analítica.


In [0]:
%sql
SELECT COUNT(*) AS total_registros FROM maintenance_silver;


total_registros
10000



A verificação confirmou a integridade dos dados, mantendo as 10.000 observações esperadas após o processamento.